In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def read_overall_comparison(file_path):
    """读取CSV文件中的Overall Comparison部分"""
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
    start_idx = None
    for i, line in enumerate(lines):
        if 'Overall Comparison' in line:
            start_idx = i
            break
    
    if start_idx is None:
        return None
    
    data = pd.read_csv(file_path, skiprows=start_idx+1)
    # 清理success rate列中的百分号
    data['CBS Success Rate'] = data['CBS Success Rate'].str.rstrip('%').astype(float)
    data['JPSCBS Success Rate'] = data['JPSCBS Success Rate'].str.rstrip('%').astype(float)
    return data

def plot_metrics_for_map(data, map_name, output_dir):
    """为单个地图绘制所有指标的对比图"""
    metrics = [
        {
            'name': 'Success Rate',
            'cbs_col': 'CBS Success Rate',
            'jpscbs_col': 'JPSCBS Success Rate',
            'ylabel': 'Success Rate (%)',
            'filename': 'success_rate'
        },
        {
            'name': 'Average Runtime',
            'cbs_col': 'CBS Avg Runtime',
            'jpscbs_col': 'JPSCBS Avg Runtime',
            'ylabel': 'Average Runtime (s)',
            'filename': 'avg_runtime'
        },
        {
            'name': 'Average Nodes',
            'cbs_col': 'CBS Avg Nodes',
            'jpscbs_col': 'JPSCBS Avg Nodes',
            'ylabel': 'Average Nodes Expanded',
            'filename': 'avg_nodes'
        }
    ]
    
    for metric in metrics:
        plt.figure(figsize=(10, 6))
        
        # 对于运行时间和节点数，只选择两个算法都有效的数据点
        if metric['name'] in ['Average Runtime', 'Average Nodes']:
            valid_mask = (data[metric['cbs_col']] > 0) & (data[metric['jpscbs_col']] > 0)
            plot_data = data[valid_mask]
        else:
            plot_data = data
            
        if len(plot_data) > 0:  # 确保有数据可画
            plt.plot(plot_data['Agents'], 
                    plot_data[metric['cbs_col']], 
                    label='CBS', 
                    marker='o',
                    linestyle='-',
                    color='blue',
                    alpha=0.7)
            
            plt.plot(plot_data['Agents'], 
                    plot_data[metric['jpscbs_col']], 
                    label='JPSCBS', 
                    marker='s',
                    linestyle='--',
                    color='red',
                    alpha=0.7)
            
            plt.xlabel('Number of Agents', fontsize=12)
            plt.ylabel(metric['ylabel'], fontsize=12)
            plt.title(f'{metric["name"]} Comparison - {map_name}', fontsize=14)
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.legend()
            
            plt.tight_layout()
            output_path = os.path.join(output_dir, f'{metric["filename"]}_{map_name}.png')
            plt.savefig(output_path, dpi=300)
        
        plt.close()

def main():
    # 获取项目根目录
    current_dir = os.getcwd()
    while not os.path.exists(os.path.join(current_dir, 'data')):
        current_dir = os.path.dirname(current_dir)
        if not current_dir:
            raise RuntimeError("Project root directory not found")
    
    results_dir = os.path.join(current_dir, 'benchmark_results')
    plots_dir = os.path.join(results_dir, 'plots')
    
    # 创建输出目录
    os.makedirs(plots_dir, exist_ok=True)
    
    # 处理每个地图的数据
    for file in os.listdir(results_dir):
        if file.startswith('benchmark_comparison_') and file.endswith('.csv'):
            map_name = file[len('benchmark_comparison_'):-4]
            file_path = os.path.join(results_dir, file)
            
            data = read_overall_comparison(file_path)
            if data is not None:
                # 为每个地图生成所有指标的对比图
                plot_metrics_for_map(data, map_name, plots_dir)
                print(f"已生成 {map_name} 的所有指标对比图")

if __name__ == "__main__":
    main()

已生成 Berlin_1_256 的所有指标对比图
已生成 Boston_0_256 的所有指标对比图
已生成 brc202d 的所有指标对比图
已生成 den312d 的所有指标对比图
已生成 den520d 的所有指标对比图
已生成 empty-16-16 的所有指标对比图
已生成 empty-32-32 的所有指标对比图
已生成 empty-48-48 的所有指标对比图
已生成 empty-8-8 的所有指标对比图
已生成 ht_chantry 的所有指标对比图
已生成 ht_mansion_n 的所有指标对比图
已生成 lak303d 的所有指标对比图
已生成 lt_gallowstemplar_n 的所有指标对比图
已生成 maze-128-128-1 的所有指标对比图
已生成 maze-128-128-10 的所有指标对比图
已生成 maze-128-128-2 的所有指标对比图
已生成 maze-32-32-2 的所有指标对比图
已生成 maze-32-32-4 的所有指标对比图
已生成 orz900d 的所有指标对比图
已生成 ost003d 的所有指标对比图
已生成 Paris_1_256 的所有指标对比图
已生成 random-32-32-10 的所有指标对比图
已生成 random-32-32-20 的所有指标对比图
已生成 random-64-64-10 的所有指标对比图
已生成 random-64-64-20 的所有指标对比图
已生成 room-32-32-4 的所有指标对比图
已生成 room-64-64-16 的所有指标对比图
已生成 room-64-64-8 的所有指标对比图
已生成 warehouse-10-20-10-2-1 的所有指标对比图
已生成 warehouse-10-20-10-2-2 的所有指标对比图
已生成 warehouse-20-40-10-2-1 的所有指标对比图
已生成 warehouse-20-40-10-2-2 的所有指标对比图
已生成 w_woundedcoast 的所有指标对比图


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os

def read_map_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    width = int(lines[2].split()[1])
    height = int(lines[1].split()[1])
    
    map_index = next(i for i, line in enumerate(lines) if line.strip() == "map") + 1
    map_data = lines[map_index:map_index + height]
    
    assert len(map_data) == height and all(len(row) == width for row in map_data), "地图数据大小不匹配"
    
    grid = np.array([[0 if cell == '.' else 1 for cell in row] for row in map_data], dtype=np.uint8)
    grid = grid.T
    
    return grid

def visualize_map(grid, output_path, scale=1.0):
    """
    显示地图并保存为图片。
    
    参数:
        grid (numpy.ndarray): 地图数据
        output_path (str): 输出图片的路径
        scale (float): 放大比例
    """
    height, width = grid.shape
    
    # 根据地图大小调整图像尺寸
    fig_width = max(10, width / 50 * scale)
    fig_height = max(10, height / 50 * scale)
    
    plt.figure(figsize=(fig_width, fig_height), dpi=100 * scale)
    plt.imshow(grid, cmap='gray_r')
    plt.title(os.path.basename(output_path)[:-4])  # 使用文件名（不含扩展名）作为标题
    plt.axis("off")
    
    # 保存图片
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()  # 关闭图形，释放内存

def main():
    # 获取项目根目录
    current_dir = os.getcwd()
    while not os.path.exists(os.path.join(current_dir, 'data')):
        current_dir = os.path.dirname(current_dir)
        if not current_dir:
            raise RuntimeError("Project root directory not found")
    
    # 地图目录
    maps_dir = os.path.join(current_dir, 'data', 'mapf-map')
    
    # 创建输出目录
    output_dir = os.path.join(current_dir, 'data', 'mapf-map', 'map_visualizations')
    os.makedirs(output_dir, exist_ok=True)
    
    # 处理所有.map文件
    for file in os.listdir(maps_dir):
        if file.endswith('.map'):
            map_path = os.path.join(maps_dir, file)
            output_path = os.path.join(output_dir, file.replace('.map', '.png'))
            
            print(f"{file}")
            try:
                grid = read_map_file(map_path)
                visualize_map(grid, output_path, scale=1.0)
                print(f"已生成地图可视化: {output_path}")
            except Exception as e:
                print(f"处理地图 {file} 时出错: {str(e)}")

if __name__ == "__main__":
    main()

Berlin_1_256.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\Berlin_1_256.png
Boston_0_256.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\Boston_0_256.png
brc202d.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\brc202d.png
den312d.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\den312d.png
den520d.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\den520d.png
empty-16-16.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\empty-16-16.png
empty-32-32.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\empty-32-32.png
empty-48-48.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\empty-48-48.png
empty-8-8.map
已生成地图可视化: C:\Users\Administrator\Desktop\JPSCBS\data\mapf-map\map_visualizations\empty-8-8.png
ht_chan

In [9]:
import os

# 获取项目根目录
current_dir = os.getcwd()
while not os.path.exists(os.path.join(current_dir, 'data')):
    current_dir = os.path.dirname(current_dir)
    if not current_dir:
        raise RuntimeError("Project root directory not found")

# 地图目录
maps_dir = os.path.join(current_dir, 'data', 'mapf-map')

# LaTeX 代码输出
latex_code = []

# 处理所有.map文件
for file in sorted(os.listdir(maps_dir)):  # 按文件名排序，保持LaTeX顺序一致
    if file.endswith('.map'):
        map_name = file.replace('.map', '')  # 提取地图名称
        map_image = f"Pictures/Benchmark/Maps/{map_name}.png"
        success_rate_image = f"Pictures/Benchmark/Success_Rate/success_rate_{map_name}.png"

        # 生成 LaTeX 代码
        figure_code = f"""
% {map_name}
\\begin{{figure}}[htbp]
    \\centering
    \\begin{{subfigure}}{{0.3\\textwidth}}
        \\centering
        \\includegraphics[width=\\textwidth]{{{map_image}}}
    \\end{{subfigure}}
    \\hfill
    \\begin{{subfigure}}{{0.6\\textwidth}}
        \\centering
        \\includegraphics[width=\\textwidth]{{{success_rate_image}}}
    \\end{{subfigure}}
    \\caption{{{map_name}}}
    \\label{{fig:{map_name}}}
\\end{{figure}}
"""
        latex_code.append(figure_code)

# 将 LaTeX 代码保存到文件
output_file = os.path.join(current_dir, "map_visualizations", "generated_figures.tex")
os.makedirs(os.path.dirname(output_file), exist_ok=True)

with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(latex_code)

print(f"LaTeX 代码已生成并保存到: {output_file}")


LaTeX 代码已生成并保存到: C:\Users\Administrator\Desktop\JPSCBS\map_visualizations\generated_figures.tex
